# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
import requests
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/karenwarmbein/.secret/yelp_api.json")
api_key = keys['api_key']

term = 'gluten free'
location = 'Seattle'

#SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
            }
response = requests.get(url, headers=headers, params=url_params)
len(response.json()['businesses'])

20

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

Preliminary experiments...

In [42]:
# import pandas as pd

# keys = get_keys("/Users/karenwarmbein/.secret/yelp_api.json")
# api_key = keys['api_key']
# term = 'gluten free'
# location = 'Seattle'
# SEARCH_LIMIT = 50
# url = 'https://api.yelp.com/v3/businesses/search'
# headers = {
#         'Authorization': 'Bearer {}'.format(api_key),
 
#             }

# dict_list_of_responses = []
# #list(dict_list_of_responses)

# for i in range(6):
#     url_params = {
#                     'term': term.replace(' ', '+'),
#                     'location': location.replace(' ', '+'),
#                     'limit': SEARCH_LIMIT,
#                     'offset': i*50
#                 }
#     response = requests.get(url, headers=headers, params=url_params)
#     response_dict = response.json()
#     dict_list_of_responses.append(response_dict)
    
    
# print(dict_list_of_responses)


In [43]:
# print(response)
# print(type(response.text))
# print(response.text[:1000])
# len(response.json()['businesses'])

Turn search into a function...

In [131]:
import pandas as pd

term = 'gluten free'
location = 'Seattle'

def responses(term, location, multiplier):
    keys = get_keys("/Users/karenwarmbein/.secret/yelp_api.json")
    api_key = keys['api_key']
    SEARCH_LIMIT = 50
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {
            'Authorization': 'Bearer {}'.format(api_key),

                }

    dict_list_of_responses = []
    for i in range(multiplier):
        url_params = {
                        'term': term.replace(' ', '+'),
                        'location': location.replace(' ', '+'),
                        'limit': SEARCH_LIMIT,
                        'offset': i*50
                    }
        response = requests.get(url, headers=headers, params=url_params)
        if response.status_code == 200:
            response_dict = response.json()
            dict_list_of_responses.append(response_dict)
        else:
            print("Error: unable to retrieve hits.  Server responded with status code", response.status_code)
    
    return dict_list_of_responses

In [155]:
term = 'gluten free'
location = 'Seattle'
multiplier = 2
res_list = responses(term, location, multiplier)
len(res_list)

2

In [167]:
res0 = pd.DataFrame(pd.DataFrame(res_list).iloc[0, 0])
res1 = pd.DataFrame(pd.DataFrame(res_list).iloc[1, 0])
        
res1.append(res0)        
res1 

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,tamarind-tree-seattle,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...","{'latitude': 47.5998406645559, 'longitude': -1...",(206) 860-1404,3137.747734,E4PzUSFuduzS6tLmGgosoQ,https://s3-media2.fl.yelpcdn.com/bphoto/_-7YyN...,False,"{'address1': '1036 S Jackson St', 'address2': ...",Tamarind Tree,+12068601404,$$,4.0,1338,[],https://www.yelp.com/biz/tamarind-tree-seattle...
1,ocho-seattle,"[{'alias': 'tapas', 'title': 'Tapas Bars'}, {'...","{'latitude': 47.66849, 'longitude': -122.38734}",(206) 784-0699,6168.211089,NvTGTMkGvkODIqOlCseF3g,https://s3-media3.fl.yelpcdn.com/bphoto/0jLa9U...,False,"{'address1': '2325 NW Market St', 'address2': ...",Ocho,+12067840699,$$,4.0,577,[],https://www.yelp.com/biz/ocho-seattle?adjust_c...
2,arepa-venezuelan-kitchen-seattle,"[{'alias': 'venezuelan', 'title': 'Venezuelan'...","{'latitude': 47.6647592219734, 'longitude': -1...",(206) 556-4879,4696.522356,qzIumu1abxuNyvpH_JzI_A,https://s3-media2.fl.yelpcdn.com/bphoto/u96Sox...,False,"{'address1': '1405 NE 50th St', 'address2': ''...",Arepa Venezuelan Kitchen,+12065564879,$,4.5,272,"[pickup, delivery]",https://www.yelp.com/biz/arepa-venezuelan-kitc...
3,flying-apron-seattle,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...","{'latitude': 47.6509412, 'longitude': -122.349...",(206) 442-1115,3032.989066,NH4mvwM9nwJm420j5UvIeQ,https://s3-media2.fl.yelpcdn.com/bphoto/cgKfwc...,False,"{'address1': '3510 Fremont Ave N', 'address2':...",Flying Apron,+12064421115,$,3.5,528,[],https://www.yelp.com/biz/flying-apron-seattle?...
4,super-six-seattle,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...","{'latitude': 47.55728, 'longitude': -122.28556}",(206) 420-1201,8461.396184,TUZ8bJufEb_DmzvIOVVutA,https://s3-media3.fl.yelpcdn.com/bphoto/rfxqyS...,False,"{'address1': '3714 S Hudson St', 'address2': '...",Super Six,+12064201201,$$,4.0,605,[],https://www.yelp.com/biz/super-six-seattle?adj...
5,casco-antiguo-seattle-4,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...","{'latitude': 47.601203, 'longitude': -122.333255}",(206) 538-0400,2698.000204,aRur9R9tTXqRgqb_NZQyBA,https://s3-media2.fl.yelpcdn.com/bphoto/5z4fmE...,False,"{'address1': '115 Occidental Ave S', 'address2...",Casco Antiguo,+12065380400,$$,4.0,517,[],https://www.yelp.com/biz/casco-antiguo-seattle...
6,dead-line-seattle,"[{'alias': 'latin', 'title': 'Latin American'}...","{'latitude': 47.60114, 'longitude': -122.33379}",(206) 556-4956,2703.247903,_aLHbaIPEO0mb3KzDkRLZw,https://s3-media2.fl.yelpcdn.com/bphoto/aCxPFv...,False,"{'address1': '114 1st Ave S', 'address2': None...",Dead Line,+12065564956,$$,4.5,151,[],https://www.yelp.com/biz/dead-line-seattle?adj...
7,re-public-seattle-2,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 47.62289, 'longitude': -122.33876}",(206) 467-5300,379.011514,orqdI_NDAdHuLNAMLhHrIg,https://s3-media2.fl.yelpcdn.com/bphoto/fVzffT...,False,"{'address1': '429 Westlake Ave N', 'address2':...",Re Public,+12064675300,$$,4.0,541,[],https://www.yelp.com/biz/re-public-seattle-2?a...
8,oak-seattle-2,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...","{'latitude': 47.5764, 'longitude': -122.31001}",(206) 535-7070,5777.181316,vB0EMRG2vo7a2K3fGXDtag,https://s3-media1.fl.yelpcdn.com/bphoto/134maP...,False,"{'address1': '3019 Beacon Ave S', 'address2': ...",Oak,+12065357070,$$,4.0,248,[],https://www.yelp.com/biz/oak-seattle-2?adjust_...
9,phoenecia-seattle,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 47.56064, 'longitude': -122.38587...",,8156.804764,yMLSALRbwRNG9JsqRUAPGw,https://s3-media3.fl.yelpcdn.com/bphoto/MlRueC...,False,"{'address1': '4717 42nd Ave SW', 'address2': N...",Phoenecia,,$$,4.5,389,[restaurant_reservation],https://www.yelp.com/biz/phoenecia-seattle?adj...


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
# Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
# Your code here

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!